# Testing paquete PeeWee

En este cuaderno quiero probar las capacidades básicas de un sistema ORM (Object–Relational Mapping) en Python.

La base de datos a utilizar va a ser SQLite por ser serverless y por tener integracion directa con peewee.

In [2]:
from peewee import *

db = SqliteDatabase('people.db')

class Person(Model):
    name = CharField()
    birthday = DateField()

    class Meta:
        database = db

Creamos una clase persona con nombre y cumpleaños, declarando los atributos con objetos propios de peewee (CharField y DateField)

La subclase Meta guarda metadatos y se pueden sobreescribir, en el snippet se sobreescribe la base de datos por ejemplo.

In [3]:
class Pet(Model):
    owner = ForeignKeyField(Person, backref='pets')
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = db

En esta segunda clase creamos una relación entre clases (y tablas) mediante el objeto Peewee ForeignKeyField.

En todo PeeWee los objetos representan diversas clases SQL:
- Una clase del objeto Model es una tabla
- Un objeto acabado en Field es una columna
- La instancia de un objeto Model es una fila

Con esa jerarquia es facil crear tablas a partir de Model con atributos Field para representar columnas y sus instancias son resultados de filas de una consulta

Como el punto de un ORM es abstraer la capa de SQL de una base de datos relacional y transferirlo a objetos, PeeWee nos permite gestionar tablas de manera sencilla, como la creacion mediante sus metodos simplificados:

In [4]:
db.connect()
db.create_tables([Person, Pet])

In [5]:
from datetime import date
uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15))
uncle_bob.save() # Devuelve el número de filas modificadas, sirve para cosas como if(Model.save()) dado a que 0 = false

1

El método create aparte de crear una instancia del objeto tambien lo crea en la BBDD, para sincronizar memoria y BBDD

In [6]:
grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1))
herb = Person.create(name='Herb', birthday=date(1950, 5, 5))

cursor = db.cursor()

cursor.execute("SELECT * FROM person")

rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'Bob', '1960-01-15')
(2, 'Grandma', '1935-03-01')
(3, 'Herb', '1950-05-05')


En cambio, se puede desincronizar al modificar instancias y para sincronizar la fila hay que llamar a save:

In [7]:
grandma.name = 'Grandma L.'
grandma.save()

cursor.execute("SELECT * FROM person")

rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'Bob', '1960-01-15')
(2, 'Grandma L.', '1935-03-01')
(3, 'Herb', '1950-05-05')


Vamos a poblar las mascotas y generar relaciones entre tablas:

In [10]:
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

In [11]:
herb_fido.owner = uncle_bob
herb_fido.save()

1

PeeWee se encarga de manejar foreign keys por nosotros.

Podemos borrar instancias y filas de la BBDD con el método delete_instance():

In [12]:
herb_mittens.delete_instance()

1

Usar SQL puro con el soporte SQLite (integrado con PeeWee por eso no requiere importar) no es necesario, se puede hacer CRUD sobre los modelos:

In [9]:
for person in Person.select():
    print(person)

1
2
3


PeeWee tambien se encarga de unir consultas, pero para ello realiza consultas adicionales, este comportamiento se le llama N+1 y es mejor evitarlo usando joins:

In [13]:
query = Pet.select().where(Pet.animal_type == 'cat')
for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [14]:
query = (Pet
         .select(Pet, Person)
         .join(Person)
         .where(Pet.animal_type == 'cat'))

for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


También se pueden hacer instancias directamente sin guardar la query:

In [15]:
for pet in Pet.select().join(Person).where(Person.name == 'Bob'):
    print(pet.name)

Kitty
Fido


Y usando otros objetos como targets directamente:

In [16]:
for pet in Pet.select().where(Pet.owner == uncle_bob):
    print(pet.name)

Kitty
Fido


Como podemos ver hay muchas formas flexibles de buscar en la BBDD fuera de las normas estrictas de SQL, pero podemos seguir usando los métodos SQL:

In [17]:
for pet in Pet.select().where(Pet.owner == uncle_bob).order_by(Pet.name):
    print(pet.name)

Fido
Kitty


In [18]:
for person in Person.select().order_by(Person.birthday.desc()):
    print(person.name, person.birthday)

Bob 1960-01-15
Herb 1950-05-05
Grandma L. 1935-03-01


Podemos también combinar filtros, pero con | y & en vez de OR y AND:

In [19]:
d1940 = date(1940, 1, 1)
d1960 = date(1960, 1, 1)
query = (Person
         .select()
         .where((Person.birthday < d1940) | (Person.birthday > d1960)))

for person in query:
    print(person.name, person.birthday)

Bob 1960-01-15
Grandma L. 1935-03-01


In [20]:
d1940 = date(1940, 1, 1)
d1960 = date(1960, 1, 1)
query = (Person
         .select()
         .where(((Person.birthday < d1940) | (Person.birthday > d1960)) & (Person.name != "Bob")))

for person in query:
    print(person.name, person.birthday)

Grandma L. 1935-03-01


Y obviamente tenemos los selectores in, between y like:

In [ ]:
query = (Person
         .select()
         .where(Person.name.in_(["Bob","Charlie"]))) # "in" es una palabra reservada en Python, la funcion real es << pero tambien se puede usar "in_"

for person in query:
    print(person.name, person.birthday)

Bob 1960-01-15


In [ ]:
query = (Person
         .select()
         .where(Person.name << ["Bob","Charlie"]))

for person in query:
    print(person.name, person.birthday)

Bob 1960-01-15


In [21]:
query = (Person
         .select()
         .where(Person.birthday.between(d1940, d1960)))

for person in query:
    print(person.name, person.birthday)

Herb 1950-05-05


In [28]:
# Like es más raro, depende de los métodos Python de Strings (contains, startswith, endswith, etc.) pero acepta ** y el string con %%
query = (Person
         .select()
         .where(Person.name ** "%G%"))

for person in query:
    print(person.name, person.birthday)

Grandma L. 1935-03-01


In [29]:
query = (Person
         .select()
         .where(Person.name.startswith("H")))

for person in query:
    print(person.name, person.birthday)

Herb 1950-05-05


In [30]:
query = (Person
         .select()
         .where(Person.name.endswith(".")))

for person in query:
    print(person.name, person.birthday)

Grandma L. 1935-03-01


In [31]:
query = (Person
         .select()
         .where(Person.name.contains("ob")))

for person in query:
    print(person.name, person.birthday)

Bob 1960-01-15


Y obviamente se pueden hacer aggregates:

In [34]:
query = (Person
         .select(Person, fn.COUNT(Pet.id).alias('pet_count')) # fn.CONST usa muchas funciones nativas de SQL que no están en métodos
         .join(Pet, JOIN.LEFT_OUTER)
         .group_by(Person)
         .order_by(Person.name))

for person in query:
    print(person.name, person.pet_count, 'pets')

Bob 2 pets
Grandma L. 0 pets
Herb 1 pets


PeeWee tra el método prefetch que crea un diccionario de fondo para cachear relaciones entre tablas usando las backref de las foreign keys, esto ayuda a no crear más consultas al iterar en objetos relacionados:

In [ ]:
query = Person.select().order_by(Person.name).prefetch(Pet)
for person in query:
    print(person.name)
    for pet in person.pets: # El nuevo atributo es la backref definida en Pet.owner
        print('  *', pet.name)

Bob
  * Kitty
  * Fido
Grandma L.
Herb
  * Mittens Jr


In [36]:
db.close()

True

Para acabar, se pueden portear bases existentes a clases con herramientas como la siguiente con el plugin pwiz:

```bash
python -m pwiz -e postgresql charles_blog > blog_models.py
```